# SQLAlchemy Core

In [1]:
!pip install sqlalchemy

     |████████████████████████████████| 6.0MB 297kB/s eta 0:00:01
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.11-cp37-cp37m-linux_x86_64.whl size=1215082 sha256=5f03524df4b5bbe0c79f057679cacb37291e055c29c1fba25901aeb2a8b74c90
  Stored in directory: /home/ronan/.cache/pip/wheels/a3/67/7d/6c41104a1a08ff1a25e260d3edec3ac19203141d1aaa2f0975
Successfully built sqlalchemy


## Établir une connexion

In [14]:
from sqlalchemy import create_engine

In [26]:
engine = create_engine("sqlite:///test.db")  # la base créée plus tôt via la DB API

In [29]:
conn = engine.connect()
conn

## Exécuter des requêtes

On peut utiliser cette connexion comme une connexion DB API pour faire des requêtes « en dur » :

In [30]:
cursor = conn.execute("SELECT * FROM fruits;")
cursor.fetchall()

[(1, 'banane'),
 (2, 'poire'),
 (3, 'framboise'),
 (4, 'pomme de reinette'),
 (5, "pomme d'api")]

## Utiliser SQLAlchemy Core pour construire des requêtes

**Avantage** : on exprime ses requêtes à plus haut niveau, et SQLAlchemy s'occupe de générer le code SQL en tenant compte du dialecte et des spécificités de la base de données utilisée.

In [31]:
engine = create_engine("sqlite:///:memory:", echo=True)

### Créer une table

On peut déclarer la structure de notre table :

In [32]:
from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()

fruits_table = Table(
    "fruits",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("nom", String),
)

Puis on peut générer les requêtes pour créer les tables déclarées :

In [33]:
metadata.create_all(engine)

2019-12-04 22:55:06,705 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-12-04 22:55:06,708 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 22:55:06,712 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-12-04 22:55:06,713 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 22:55:06,715 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("fruits")
2019-12-04 22:55:06,717 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 22:55:06,718 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("fruits")
2019-12-04 22:55:06,720 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 22:55:06,721 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE fruits (
	id INTEGER NOT NULL, 
	nom VARCHAR, 
	PRIMARY KEY (id)
)


2019-12-04 22:55:06,723 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 22:55:06,725 INFO sqlalchemy.engine.base.Engine COMMIT


### Insérer des données

On peut insérer une ligne dans la table :

In [34]:
conn = engine.connect()

In [35]:
query = fruits_table.insert().values(nom="banane")
conn.execute(query)

2019-12-04 22:55:50,069 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-04 22:55:50,072 INFO sqlalchemy.engine.base.Engine ('banane',)
2019-12-04 22:55:50,074 INFO sqlalchemy.engine.base.Engine COMMIT


On peut aussi insérer plusieurs lignes d'un coup :

In [37]:
query = fruits_table.insert()
conn.execute(query, [
    {"nom": "pomme"},
    {"nom": "kiwi"},
    {"nom": "framboise"},
])

2019-12-04 23:02:31,208 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-04 23:02:31,211 INFO sqlalchemy.engine.base.Engine (('pomme',), ('kiwi',), ('framboise',))
2019-12-04 23:02:31,213 INFO sqlalchemy.engine.base.Engine COMMIT


### Faire une requête

In [40]:
from sqlalchemy import select

query = select([fruits_table])
cursor = conn.execute(query)
cursor.fetchall()

2019-12-04 23:03:44,805 INFO sqlalchemy.engine.base.Engine SELECT fruits.id, fruits.nom 
FROM fruits
2019-12-04 23:03:44,808 INFO sqlalchemy.engine.base.Engine ()


[(1, 'banane'),
 (2, 'pomme'),
 (3, 'kiwi'),
 (4, 'framboise'),
 (5, 'pomme'),
 (6, 'kiwi'),
 (7, 'framboise')]

On peut spécifier les colonnes voulues :

In [41]:
query = select([fruits_table.c.nom])
cursor = conn.execute(query)
cursor.fetchall()

2019-12-04 23:04:09,576 INFO sqlalchemy.engine.base.Engine SELECT fruits.nom 
FROM fruits
2019-12-04 23:04:09,577 INFO sqlalchemy.engine.base.Engine ()


[('banane',),
 ('pomme',),
 ('kiwi',),
 ('framboise',),
 ('pomme',),
 ('kiwi',),
 ('framboise',)]

On peut ajouter une clause `WHERE` :

In [44]:
query = select([fruits_table.c.nom]).where(fruits_table.c.nom.like("b%"))
cursor = conn.execute(query)
cursor.fetchall()

2019-12-04 23:06:01,737 INFO sqlalchemy.engine.base.Engine SELECT fruits.nom 
FROM fruits 
WHERE fruits.nom LIKE ?
2019-12-04 23:06:01,739 INFO sqlalchemy.engine.base.Engine ('b%',)


[('banane',)]

## Références

- https://docs.sqlalchemy.org/en/13/core/tutorial.html